## Import packages

### Data science toolkit

In [1]:
import numpy as np
import pandas as pd

### Machine learning

In [2]:
from spotlight.cross_validation import user_based_train_test_split
from spotlight.evaluation import *
from spotlight.interactions import Interactions
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.factorization.explicit import ExplicitFactorizationModel

In [3]:
from spotlight.factorization.representations import BilinearNet
from spotlight.layers import BloomEmbedding

In [4]:
import torch

In [5]:
from torch.optim import sparse_adam

In [6]:
from spotlight.evaluation import mrr_score

In [7]:
from scipy.stats import describe

## Reading data

In [39]:
dataset = pd.read_csv('dataset_aggr.csv')

In [40]:
user_ids = dataset.UserID.values
item_ids = dataset.ItemID.values

ratings = dataset.avg_rating.values

# num_users = len(set(user_ids)) - set automaticly
# num_items = len(set(item_ids))

In [41]:
interactions = Interactions(user_ids = np.array(user_ids, dtype=np.int32),
                            item_ids = np.array(item_ids, dtype=np.int32),
                            ratings = np.array(ratings, dtype=np.float32))

In [42]:
train, test = user_based_train_test_split(interactions, test_percentage=0.20)

### Check train-test proportions

In [43]:
train, test

(<Interactions dataset (1043 users x 763 items x 656 interactions)>,
 <Interactions dataset (1043 users x 763 items x 274 interactions)>)

## Train "implicit factorization model"

### Define parameters

In [44]:
model = ImplicitFactorizationModel(loss='adaptive_hinge', 
                                   embedding_dim=128, 
                                   n_iter=100, 
                                   batch_size=32,
                                   learning_rate=0.005,
                                   l2=1e-6,
                                   optimizer_func=sparse_adam.SparseAdam,
                                   sparse=True,
                                   num_negative_samples=10)

# It takes around 1-3 minutes on a CPU

### Fit model

In [45]:
model.fit(train, verbose=False)

# This should take no more than 1 minute

### Average MRR

In [46]:
mrr = mrr_score(model, test)

In [47]:
avg_mrr = mrr.mean()

In [48]:
f'Avg mrr score (for test data) is {avg_mrr}'

'Avg mrr score (for test data) is 0.026739688722935646'

### RMSE score

In [49]:
rmse = rmse_score(model, test)

In [50]:
f'RMSE score (for test data) is {rmse}'

'RMSE score (for test data) is 2.5198733806610107'

### Precision-recall at k

#### On the train set

In [51]:
precision, recall = precision_recall_score(model, train)

In [52]:
print(f'Precision per user: {precision}')

Precision per user: [1.  1.  1.  0.5 1.  1.  0.6 1.  1.  0.5 0.2 1.  0.5 0.5 0.1 1.  1.  1.
 1.  0.4 1.  0.3 0.5 1.  0.5 1.  1.  0.1 1.  1.  1.  1. ]


In [53]:
describe(precision)

DescribeResult(nobs=32, minmax=(0.1, 1.0), mean=0.771875, variance=0.10015120967741935, skewness=-0.8668884123413311, kurtosis=-0.7722939722185149)

In [54]:
print(f'Recall per user: {recall}')

Recall per user: [1.         0.33333333 0.83333333 1.         0.47619048 0.1010101
 1.         0.4        0.14285714 1.         1.         0.66666667
 1.         1.         1.         0.66666667 0.4        1.
 0.25       1.         0.5        1.         1.         1.
 1.         0.5        0.0862069  1.         0.5        0.4
 0.625      1.        ]


In [55]:
describe(recall)

DescribeResult(nobs=32, minmax=(0.08620689655172414, 1.0), mean=0.7150395192690451, variance=0.10585036113274789, skewness=-0.5615055670941068, kurtosis=-1.161261458133152)

#### On the test set

In [56]:
precision, recall = precision_recall_score(model, test)

In [57]:
print(f'Precision per user: {precision}')

Precision per user: [0.  0.3 1.  0.  0.1 0.  0.  0.1 0.  0. ]


In [58]:
describe(precision)

DescribeResult(nobs=10, minmax=(0.0, 1.0), mean=0.15, variance=0.09833333333333333, skewness=2.2675633295781363, kurtosis=3.7102365220722033)

In [59]:
print(f'Recall per user: {recall}')

Recall per user: [0.         0.04285714 0.07194245 0.         0.1        0.
 0.         0.2        0.         0.        ]


In [60]:
describe(recall)

DescribeResult(nobs=10, minmax=(0.0, 0.2), mean=0.04147995889003084, variance=0.004422953371262761, skewness=1.5066110907220818, kurtosis=1.1765322639546207)

## Train "explicit factorization model"

In [61]:
model = ExplicitFactorizationModel(loss='poisson', 
                                   embedding_dim=128, 
                                   n_iter=150, 
                                   batch_size=32,
                                   learning_rate=0.005,
                                   l2=1e-6,
                                   optimizer_func=sparse_adam.SparseAdam,
                                   sparse=True)

### Fit model

In [62]:
model.fit(train, verbose=False)

# It takes around 1-3 minutes on a CPU

### Average MRR

In [63]:
mrr = mrr_score(model, test)

In [64]:
avg_mrr = mrr.mean()

In [65]:
f'Avg mrr score (for test data) is {avg_mrr}'

'Avg mrr score (for test data) is 0.037557012727262064'

### RMSE score

In [66]:
rmse = rmse_score(model, test)

In [67]:
f'RMSE score (for test data) is {rmse}'

'RMSE score (for test data) is 1.7787281274795532'

### Precision-recall at k

In [68]:
precision, recall = precision_recall_score(model, test)

In [69]:
print(f'Precision per user: {precision}')
print(describe(precision))
print(f'Recall per user: {recall}')
print(describe(recall))

Precision per user: [0.  0.5 1.  0.  0.  0.1 0.1 0.2 0.  0. ]
DescribeResult(nobs=10, minmax=(0.0, 1.0), mean=0.19, variance=0.10544444444444447, skewness=1.797450224429637, kurtosis=1.9561315166205677)
Recall per user: [0.         0.07142857 0.07194245 0.         0.         0.1
 0.2        0.4        0.         0.        ]
DescribeResult(nobs=10, minmax=(0.0, 0.4), mean=0.0843371017471737, variance=0.016572254338652115, skewness=1.6470441745565685, kurtosis=1.6574059636006355)


## Model bloom embeddings

In [70]:
compression_ratio = 1.5

In [71]:
user_embeddings = BloomEmbedding(interactions.num_users, 32,
                                 compression_ratio=compression_ratio,
                                 num_hash_functions=2)

item_embeddings = BloomEmbedding(interactions.num_items, 32,
                                 compression_ratio=compression_ratio,
                                 num_hash_functions=2)

network = BilinearNet(interactions.num_users,
                      interactions.num_items,
                      user_embedding_layer=user_embeddings,
                      item_embedding_layer=item_embeddings)

model = ExplicitFactorizationModel(loss='poisson',
                                   n_iter=150,
                                   batch_size=32,
                                   learning_rate=1e-2,
                                   l2=1e-6,
                                   representation=network,
                                   sparse=True,
                                   use_cuda=False)

In [72]:
model.fit(train)

In [73]:
mrr = mrr_score(model, test, train=train)
f'Avg mrr score is {np.mean(mrr)}'

'Avg mrr score is 0.028275384158525645'

In [74]:
rmse = rmse_score(model, test)
f'RMSE score (for test data) is {rmse}'

'RMSE score (for test data) is 1.6212029457092285'

In [75]:
precision, recall = precision_recall_score(model, test, train=train)

In [76]:
print(f'Precision per user: {precision}')
print(describe(precision))
print(f'Recall per user: {recall}')
print(describe(recall))

Precision per user: [0.  0.7 1.  0.  0.  0.1 0.1 0.  0.  0. ]
DescribeResult(nobs=10, minmax=(0.0, 1.0), mean=0.19, variance=0.1276666666666667, skewness=1.5956848217809907, kurtosis=0.8332776599926821)
Recall per user: [0.         0.1        0.07194245 0.         0.         0.1
 0.2        0.         0.         0.        ]
DescribeResult(nobs=10, minmax=(0.0, 0.2), mean=0.04719424460431655, variance=0.004766972033883685, skewness=1.1554269614124637, kurtosis=0.21025845544285238)


## Unaggregated set, explicit model (with ratings)

In [77]:
dataset = pd.read_csv('dataset.csv')

In [78]:
user_ids = dataset.UserID.values
item_ids = dataset.ItemID.values

ratings = dataset.Rating.values

interactions = Interactions(user_ids = np.array(user_ids, dtype=np.int32),
                            item_ids = np.array(item_ids, dtype=np.int32),
                            ratings = np.array(ratings, dtype=np.float32))

train, test = user_based_train_test_split(interactions, test_percentage=0.20)

In [79]:
train, test 

(<Interactions dataset (1043 users x 763 items x 3808 interactions)>,
 <Interactions dataset (1043 users x 763 items x 204 interactions)>)

In [80]:
model = ExplicitFactorizationModel(loss='poisson', 
                                   embedding_dim=256, 
                                   n_iter=200, 
                                   batch_size=32,
                                   learning_rate=0.005,
                                   l2=1e-6)

In [81]:
model.fit(train)

# This can take up to 10 minutes on CPU

In [82]:
mrr = mrr_score(model, test, train=train)
f'Avg mrr score is {np.mean(mrr)}'

'Avg mrr score is 0.029043451390917196'

In [83]:
rmse = rmse_score(model, test)
f'RMSE score (for test data) is {rmse}'

'RMSE score (for test data) is 1.6366691589355469'

In [84]:
precision, recall = precision_recall_score(model, test, train=train)

In [85]:
print(f'Precision per user: {precision}')
print(describe(precision))
print(f'Recall per user: {recall}')
print(describe(recall))

Precision per user: [0.2 0.3 0.  0.  0. ]
DescribeResult(nobs=5, minmax=(0.0, 0.3), mean=0.1, variance=0.020000000000000004, skewness=0.5929270612815708, kurtosis=-1.4375000000000009)
Recall per user: [0.0952381 0.3       0.        0.        0.       ]
DescribeResult(nobs=5, minmax=(0.0, 0.3), mean=0.07904761904761905, variance=0.016956916099773244, skewness=1.1784018350520786, kurtosis=-0.28231835403999783)


## Conclusions

Despite no user and item features used by the algorithms, we managed to get 1.5 RMSE and 20% precision at k-10. For RMSE, the performance is below algorithms like ALS or SVD. Yet, precision was closer to current state of art algorithms like SAR or NCF. Reference - https://github.com/microsoft/recommenders